In [ ]:
!rm -rf loracraft
!git clone https://github.com/PPierzc/loracraft.git
!mv loracraft loracraft-git
!mv loracraft-git/* .
!rm -rf loracraft-git

In [ ]:
!pip install -r requirements.txt

In [ ]:
import json
import torch
import os
from loracraft.pipeline import get_pipeline
from loracraft.generator_utils import save_image

In [ ]:
with open ("/content/lora_db.json", 'r') as f:
    lora_db = json.load(f)

In [ ]:
config = {
    "loras": ["SewingDoll", "CinematicPainting"],
    "weights": [0.7, 0.3],
    "prompts": ["Test shit"],
    "trials": 1,

    'images_path': '/content/output/sewingdoll-cinematic/',
    'seed': 42,
    'pipeline_kwargs': {
        'num_inference_steps': 1,
        "guidance_scale": 0,
    }
}

In [ ]:
## create pipeline
pipe = get_pipeline()

In [ ]:
# Download required LoRAs
for lora in config["loras"]:

    lora_path = f"{lora}.safetensors"
    lora_wget = lora_db[lora]["wget_link"]

    if not os.path.exists(lora_path):
        ! wget $lora_wget -O $lora_path

    pipe.load_lora_weights(
        lora_path,
        adapter_name=lora
    )

In [ ]:
pipe.set_adapters(config["loras"], adapter_weights=config["weights"])

In [ ]:
for trial in range(config['trials']):
    for prompt in config['prompts']:
        image = pipe(
            prompt=prompt,
            generator=torch.manual_seed(config['seed']),
            **config['pipeline_kwargs']
        ).images[0]
        save_image(image, config, prompt=prompt, trial=trial)